In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

path = "C:/Users/Will/Downloads/dataset/"

upvote_test = pd.read_csv(path + "train_NIR5Yl1.csv")

upvote_train_url = "https://raw.githubusercontent.com/tcy98/colabDataStoreage/main/test_upvote.csv"

upvote_test.head(10)

ID Tag  Reputation  Answers  Username    Views  Upvotes
0   52664   a      3942.0      2.0    155623   7855.0     42.0
1  327662   a     26046.0     12.0     21781  55801.0   1175.0
2  468453   c      1358.0      4.0     56177   8067.0     60.0
3   96996   a       264.0      3.0    168793  27064.0      9.0
4  131465   c      4271.0      4.0    112223  13986.0     83.0
5  345568   r       440.0      1.0     23866   5577.0      4.0
6  376865   c       147.0      1.0     10759  17184.0     17.0
7  467378   j      2269.0      2.0     54623    312.0      3.0
8  393640   j       111.0      2.0    172926  53738.0     79.0
9  130054   c      2749.0      1.0     10189    227.0      0.0

In [2]:
#check to see if there are missing values 

upvote_test.isna().sum()

ID            0
Tag           0
Reputation    0
Answers       0
Username      0
Views         0
Upvotes       0
dtype: int64

In [3]:
from sklearn.preprocessing import OrdinalEncoder


#OrdinalEncoder 
enc = OrdinalEncoder()
upvote_Tag = upvote_test["Tag"]
upvote_Tag.head(10)


upvote_Tag2= upvote_test["Tag"].values.reshape(-1,1)
upvote_Tag2

upvote_test["Tag"] = enc.fit_transform(upvote_Tag2)




In [5]:
train_y = upvote_test["Upvotes"]

primary_list   = ['Tag', 'Reputation', 'Answers', 'Username', 'Views']
train1 = upvote_test[primary_list]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train1.values, train_y.values, test_size = 0.2, random_state = 7)

# LINEAR REGRESSION

In [6]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [7]:
ypred = lin_reg.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': ypred})
df

Actual    Predicted
0        11.0  -336.518548
1        11.0  -400.414876
2        94.0   -94.430556
3       207.0    49.303697
4        66.0   -64.031968
...       ...          ...
66004   821.0  4545.862515
66005     5.0   939.105634
66006     2.0  -399.681642
66007     6.0  -518.755530
66008     7.0  -292.486942

[66009 rows x 2 columns]

# Decision Tree Regressor

In [8]:

# import the regressor
from sklearn.tree import DecisionTreeRegressor 
  
# create a regressor object
regressor = DecisionTreeRegressor(random_state = 0) 
  
# fit the regressor with X and Y data
regressor.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

# Mean Squared Error Decision Tree Regressor 

In [32]:
mse = mean_squared_error(df['Actual'], df['Predicted'])
print(mse)

1658423.5264433639


In [33]:
from sklearn.metrics import mean_squared_error

predictions = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
df.head(20)



Actual  Predicted
0     11.0        6.0
1     11.0        0.0
2     94.0       56.0
3    207.0      112.0
4     66.0       75.0
5     61.0       28.0
6    130.0      120.0
7    175.0      187.0
8    436.0      515.0
9     82.0      158.0
10  3539.0     6583.0
11     0.0        8.0
12    13.0        9.0
13    12.0        4.0
14     4.0        3.0
15     0.0        5.0
16    88.0       50.0
17    59.0       28.0
18    50.0       41.0
19    49.0       82.0

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier(random_state=0)
decision_tree_result = cross_validation(decision_tree_model, train1, train_y, 5)
print(decision_tree_result)

# K Fold Cross Validation

In [62]:
model = DecisionTreeRegressor(random_state=0)
accuracy = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv = 10)
print(accuracy)
#get the mean of each fold 
print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100)

[ -3429434.93523709  -2344953.58650204  -1403275.96977731
  -1845858.51424027 -12396121.97924557  -1422107.26564157
  -1093201.83827595  -1525388.4016589   -2037135.6902246
  -4375581.54815741]
Accuracy of Model with Cross Validation is: -318730597.28960705


# Ensemble of Method

In [66]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingRegressor
#
# Pipeline Estimator
#
pipeline = make_pipeline(StandardScaler(),
                        LogisticRegression(random_state=1))
#
# Instantiate the bagging classifier
#
bgclassifier = BaggingRegressor(base_estimator=pipeline, n_estimators=100,
                                 max_features=0.5,
                                 max_samples=100,
                                 random_state=1, n_jobs=5)
#
# Fit the bagging classifier
#
bgclassifier.fit(X_train, y_train)
#
# Model scores on test and training data
#
print('Model test Score: %.3f, ' %bgclassifier.score(X_test, y_test),
      'Model training Score: %.3f' %bgclassifier.score(X_train, y_train))

Model test Score: 0.099,  Model training Score: 0.068


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {"criterion": ["mse", "mae"],
              "min_samples_split": [10, 20, 40],
              "max_depth": [2, 6, 8],
              "min_samples_leaf": [20, 40, 100],
              "max_leaf_nodes": [5, 20, 100],
              }

base_cls = DecisionTreeRegressor()

grid_search = GridSearchCV(base_cls, params, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(X_train, y_train)